In [1]:
# Useful additional packages 
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from math import pi


from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit import BasicAer, execute
from qiskit.quantum_info import Pauli, state_fidelity, basis_state, process_fidelity 

from qiskit.circuit import Parameter


from qiskit import *
IBMQ.load_accounts(hub=None)
from qiskit.tools.monitor import job_monitor, backend_monitor, backend_overview
from qiskit.providers.ibmq import least_busy

shots = 8192 
max_credits = 3 
#job_exp = execute(qc, backend=backend, shots=shots, max_credits=max_credits)
from qiskit.compiler import transpile

In [2]:
#Simulation
#Misura di tutti i qubit
n=5
q = QuantumRegister(n, 'q')
c = ClassicalRegister(n, 'c') #n

circ = QuantumCircuit(q,c)
phi = Parameter('phi')

circ.h(q[0])

for i in range(n-1):
    circ.cx(i, i+1)
    
circ.barrier()
circ.rz(phi, range(n))
circ.barrier()

for i in reversed(range(n-1)):
    circ.cx(i, i+1)

circ.h(q[0])

circ.barrier()

for i in range(n):
    circ.measure(q[i], c[i])
    


#circ.draw(output='mpl')

print('Phi range')
phi_range = [np.pi*j/(n+1) for j in range(2*n+1+1)]
print(phi_range)
circuits = [circ.bind_parameters({phi: phi_val}) for phi_val in phi_range] 
print(circuits[-1].parameters)
circuits[-1].draw(output='mpl')

#circuits[1].draw()


job = execute(circ,
            backend=BasicAer.get_backend('qasm_simulator'),
            parameter_binds=[{phi: phi_val}
              for phi_val in phi_range])

phi_range2 = phi_range
counts2 =[job.result().get_counts(i) for i in range(len(job.result().results))] 

print('\n')
print('Counts')
print(counts2)
print('\n')
print('Counts of 00000')
print(list(map(lambda c: 1/1024*c.get('00000', 0), counts2)))

Phi range
[0.0, 0.5235987755982988, 1.0471975511965976, 1.5707963267948966, 2.0943951023931953, 2.6179938779914944, 3.141592653589793, 3.665191429188092, 4.1887902047863905, 4.71238898038469, 5.235987755982989, 5.759586531581287]
set()


Counts
[{'00000': 1024}, {'00001': 947, '00000': 77}, {'00001': 271, '00000': 753}, {'00001': 511, '00000': 513}, {'00001': 761, '00000': 263}, {'00001': 58, '00000': 966}, {'00001': 1024}, {'00001': 71, '00000': 953}, {'00001': 776, '00000': 248}, {'00001': 518, '00000': 506}, {'00001': 278, '00000': 746}, {'00001': 963, '00000': 61}]


Counts of 00000
[1.0, 0.0751953125, 0.7353515625, 0.5009765625, 0.2568359375, 0.943359375, 0.0, 0.9306640625, 0.2421875, 0.494140625, 0.728515625, 0.0595703125]


In [3]:
#Run in real quantum computer
#Misura di tutti i qubit
n = 5
q = QuantumRegister(n, 'q')
c = ClassicalRegister(n, 'c') #n

circ = QuantumCircuit(q,c)
phi = Parameter('phi')

circ.h(q[0])

for i in range(n-1):
    circ.cx(i, i+1)
    
circ.barrier()
circ.rz(phi, range(n))
circ.barrier()

for i in reversed(range(n-1)):
    circ.cx(i, i+1)

circ.h(q[0])

circ.barrier()

for i in range(n):
    circ.measure(q[i], c[i]) 

#circ.draw(output='mpl')

print('Phi range')
phi_range = [np.pi*j/(n+1) for j in range(2*n+1+1)]
print(phi_range)
circuits = [circ.bind_parameters({phi: phi_val}) for phi_val in phi_range] 
#print(circuits[3].parameters)
#circuits[11].draw(output='mpl')


job = execute(circ,
            backend=IBMQ.get_backend('ibmqx2'),
            parameter_binds=[{phi: phi_val}
              for phi_val in phi_range],shots=shots)

job_monitor(job)

phi_range2 = phi_range
counts2 =[job.result().get_counts(i) for i in range(len(job.result().results))] 

print('\n')
print('Counts')
print(counts2)
print('\n')
print('Counts of 00000')
print(list(map(lambda c: 1/shots*c.get('00000', 0), counts2)))
job_id = job.job_id()
print('JOB ID: {}'.format(job_id))

Phi range
[0.0, 0.5235987755982988, 1.0471975511965976, 1.5707963267948966, 2.0943951023931953, 2.6179938779914944, 3.141592653589793, 3.665191429188092, 4.1887902047863905, 4.71238898038469, 5.235987755982989, 5.759586531581287]
Job Status: job has successfully run


Counts
[{'11011': 13, '11000': 69, '11100': 21, '01100': 203, '01110': 80, '01011': 72, '00001': 2581, '00101': 587, '01000': 306, '10101': 23, '10000': 53, '11110': 14, '01010': 98, '01001': 457, '00000': 1362, '10010': 11, '00110': 128, '01111': 87, '00100': 662, '11111': 16, '00111': 105, '10110': 5, '11010': 11, '11001': 104, '00011': 276, '10100': 16, '11101': 59, '00010': 283, '10001': 115, '10011': 8, '01101': 361, '10111': 6}, {'11011': 14, '11000': 53, '11100': 29, '01100': 332, '01110': 58, '01011': 49, '00001': 1333, '00101': 831, '01000': 336, '10101': 24, '10000': 72, '11110': 9, '01010': 46, '01001': 270, '00000': 2497, '10010': 5, '00110': 188, '01111': 68, '00100': 682, '11111': 14, '00111': 173, '10110': 

In [4]:
#Run in real quantum computer - Transpile
#Misura di tutti i qubit
n = 5
q = QuantumRegister(n, 'q')
c = ClassicalRegister(n, 'c') #n

circ = QuantumCircuit(q,c)
phi = Parameter('phi')

circ.h(q[0])

for i in range(n-1):
    circ.cx(i, i+1)
    
circ.barrier()
circ.rz(phi, range(n))
circ.barrier()

for i in reversed(range(n-1)):
    circ.cx(i, i+1)

circ.h(q[0])

circ.barrier()

for i in range(n):
    circ.measure(q[i], c[i]) 

#circ.draw(output='mpl')

print('Phi range')
phi_range = [np.pi*j/(n+1) for j in range(2*n+1+1)]
print(phi_range)
circuits = [circ.bind_parameters({phi: phi_val}) for phi_val in phi_range] 
#print(circuits[3].parameters)
#circuits[11].draw(output='mpl')

backend = IBMQ.get_backend('ibmqx2')
trans_ghz = transpile(circ, backend)

job = execute(circ,
            backend,
            parameter_binds=[{phi: phi_val}
              for phi_val in phi_range],shots=shots)

job_monitor(job)

phi_range2 = phi_range
counts2 =[job.result().get_counts(i) for i in range(len(job.result().results))] 

print('\n')
print('Counts')
print(counts2)
print('\n')
print('Counts of 00000')
print(list(map(lambda c: 1/shots*c.get('00000', 0), counts2)))

job_id = job.job_id()
print('JOB ID: {}'.format(job_id))

#retrieved_job = backend.retrieve_job(job_id)
#retrieved_job.result().get_counts(qc)

Phi range
[0.0, 0.5235987755982988, 1.0471975511965976, 1.5707963267948966, 2.0943951023931953, 2.6179938779914944, 3.141592653589793, 3.665191429188092, 4.1887902047863905, 4.71238898038469, 5.235987755982989, 5.759586531581287]
Job Status: job has successfully run


Counts
[{'11011': 15, '11000': 59, '11100': 23, '01100': 184, '01110': 89, '01011': 68, '00001': 2505, '00101': 579, '01000': 332, '10101': 22, '10000': 58, '11110': 8, '01010': 78, '01001': 457, '00000': 1467, '10010': 13, '00110': 152, '01111': 73, '00100': 686, '11111': 6, '00111': 111, '10110': 7, '11010': 8, '11001': 90, '00011': 255, '10100': 21, '11101': 51, '00010': 302, '10001': 92, '10011': 8, '01101': 367, '10111': 6}, {'11011': 19, '11000': 78, '11100': 53, '01100': 335, '01110': 71, '01011': 44, '00001': 1344, '00101': 878, '01000': 292, '10101': 22, '10000': 69, '11110': 10, '01010': 57, '01001': 321, '00000': 2348, '10010': 12, '00110': 180, '01111': 79, '00100': 648, '11111': 15, '00111': 161, '10110': 6, 

In [5]:
#Run in real quantum computer - Transpile user
#Misura di tutti i qubit
n = 5
q = QuantumRegister(n, 'q')
c = ClassicalRegister(n, 'c') #n

circ = QuantumCircuit(q,c)
phi = Parameter('phi')

circ.h(q[0])

for i in range(n-1):
    circ.cx(i, i+1)
    
circ.barrier()
circ.rz(phi, range(n))
circ.barrier()

for i in reversed(range(n-1)):
    circ.cx(i, i+1)

circ.h(q[0])

circ.barrier()

for i in range(n):
    circ.measure(q[i], c[i]) 

#circ.draw(output='mpl')

print('Phi range')
phi_range = [np.pi*j/(n+1) for j in range(2*n+1+1)]
print(phi_range)
circuits = [circ.bind_parameters({phi: phi_val}) for phi_val in phi_range] 
#print(circuits[3].parameters)
#circuits[11].draw(output='mpl')

layout = {q[0]: 4, q[1]: 3, q[2]: 2, q[3]:1, q[4]:0}

backend = IBMQ.get_backend('ibmqx2')
trans_ghz_mapped = transpile(circ, backend, initial_layout=layout)

job = execute(circ,
            backend,
            parameter_binds=[{phi: phi_val}
              for phi_val in phi_range],shots=shots)

job_monitor(job)

phi_range2 = phi_range
counts2 =[job.result().get_counts(i) for i in range(len(job.result().results))] 

print('\n')
print('Counts')
print(counts2)
print('\n')
print('Counts of 00000')
print(list(map(lambda c: 1/shots*c.get('00000', 0), counts2)))

job_id = job.job_id()
print('JOB ID: {}'.format(job_id))

#retrieved_job = backend.retrieve_job(job_id)
#retrieved_job.result().get_counts(qc)

Phi range
[0.0, 0.5235987755982988, 1.0471975511965976, 1.5707963267948966, 2.0943951023931953, 2.6179938779914944, 3.141592653589793, 3.665191429188092, 4.1887902047863905, 4.71238898038469, 5.235987755982989, 5.759586531581287]
Job Status: job has successfully run


Counts
[{'11011': 19, '11000': 60, '11100': 34, '01100': 183, '01110': 94, '01011': 76, '00001': 2363, '00101': 584, '01000': 307, '10101': 29, '10000': 51, '11110': 13, '01010': 73, '01001': 457, '00000': 1507, '10010': 10, '00110': 154, '01111': 84, '00100': 677, '11111': 13, '00111': 155, '10110': 6, '11010': 16, '11001': 85, '00011': 289, '10100': 19, '11101': 51, '00010': 339, '10001': 90, '10011': 11, '01101': 338, '10111': 5}, {'11011': 5, '11000': 71, '11100': 27, '01100': 303, '01110': 73, '01011': 48, '00001': 1421, '00101': 847, '01000': 309, '10101': 29, '10000': 67, '11110': 14, '01010': 49, '01001': 268, '00000': 2363, '10010': 15, '00110': 210, '01111': 66, '00100': 673, '11111': 12, '00111': 149, '10110': 